In [1]:
import sys
import os
import pickle
import numpy as np
sys.path.append(os.pardir)
from mnist import load_mnist

In [2]:
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=True, one_hot_label=False)
    # load_mnist -> (訓練画像, 訓練ラベル), (テスト画像, テストラベル)  という形式でMNISTデータを返す
    # 引数  -> normalize: 入力画像を0.0~1.0の値に正規化するかどうかを設定。Falseの場合入力画像は元の0~255
    #       -> flatten: 入力画像を平らにするかどうか、Falseの場合1x28x28、Trueの場合784個の要素からなる1次元配列
    return x_test, t_test

def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    # pickleファイルのsample_weight.pklに保存された学習済みの重みパラメーターを読み込む。
    # このファイルには重みとバイアスのパラメーターがdict型の変数として保存されている。
    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid_func(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid_func(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax_func(a3)
    return y

def sigmoid_func(x):
    return 1 / (1 + np.exp(-x))

def identify_func(x):
    return x

def softmax_func(a):
    c = np.max(a)
    exp_a = np.exp(a - c)   # オーバーフロー対策
    sum_exp_a = np.sum(exp_a)
    return exp_a / sum_exp_a


In [5]:
x, t = get_data()
network = init_network()

accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])  # 各ラベルの確率がNumpy配列として出力される。
    #print(y)
    p = np.argmax(y)            # 最も確率の高い要素(値の最大値)のインデックスを取得
    print(p)
    if p == t[i]:
        accuracy_cnt += 1
# 最後にニューラルネットワークが予測した答えと正解ラベルとを比較して正解した割合を認識精度とする。
print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

7
2
1
0
4
1
4
9
6
9
0
6
9
0
1
5
9
7
3
4
9
6
6
5
4
0
7
4
0
1
3
1
3
6
7
2
7
1
2
1
1
7
4
2
3
5
1
2
4
4
6
3
5
5
6
0
4
1
9
5
7
8
9
3
7
4
2
4
3
0
7
0
2
9
1
7
3
2
9
7
7
6
2
7
8
4
7
3
6
1
3
6
4
3
1
4
1
7
6
9
6
0
5
4
9
9
2
1
9
4
8
7
3
9
7
4
4
4
9
2
5
4
7
6
4
9
0
5
8
5
6
6
5
7
8
1
0
1
6
4
6
7
3
1
7
1
8
2
0
9
9
9
5
5
1
5
6
0
3
4
4
6
5
4
6
5
4
5
1
4
4
7
2
3
2
7
1
8
1
8
1
8
5
0
8
9
2
5
0
1
1
1
0
9
0
3
1
6
4
2
3
6
1
1
1
3
9
5
2
9
4
5
9
3
9
0
3
5
5
5
7
2
2
7
1
2
8
4
1
7
3
3
8
7
7
9
2
2
4
1
5
8
8
7
2
5
0
2
4
2
4
1
9
5
7
7
2
8
2
0
8
5
7
7
9
1
8
1
8
0
3
0
1
9
9
4
1
8
2
1
2
9
7
5
9
2
6
4
1
5
4
2
9
2
0
4
0
0
2
8
6
7
1
2
4
0
2
7
4
3
3
0
0
5
1
9
6
5
2
5
7
7
9
3
0
4
2
0
7
1
1
2
1
5
3
3
9
7
8
6
3
4
1
3
8
1
0
5
1
3
1
5
0
6
1
8
5
1
9
9
4
6
7
2
5
0
6
5
6
3
7
2
0
8
8
5
4
1
1
4
0
7
3
7
6
1
6
2
1
9
2
8
6
1
9
5
2
5
4
4
2
8
3
8
2
4
5
0
3
1
7
7
5
7
9
7
1
9
2
1
4
2
9
2
0
4
9
1
4
8
1
8
4
5
9
8
8
3
7
6
0
0
3
0
2
0
6
4
8
5
3
3
2
3
9
1
2
6
8
0
5
6
6
6
3
8
8
2
7
5
8
9
6
1
8
4
1
2
8
3
1
9
7
5
4
0
8
9
9
1
0
5
2
3
7
0
9
4
0
6


## バッチ処理
多次元配列の対応する次元の要素数が一致する。最終的な結果として要素数が10の一次元配列yが出力される。\
入力データの形状は10000x784、出力データの形状は100x10になっている。\
これは、100枚分の入力データの結果が一度に出力されることを表している\
まとまりのある入力データをbatchと呼ぶ

In [4]:
x, t = get_data()
network = init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3']

# 形状の確認
print(x.shape)
print(W1.shape)
print(W2.shape)
print(W3.shape)
# ---------

batch_size = 100 # batchサイズ
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)      # 一次元目の要素ごとに最大値のインデックスを取得
    # 最後にバッチ単位で分類した結果と実際の答えを比較する。
    # そのためにnumpy配列同士でboolean配列を作成しTrueの個数を算出している
    accuracy_cnt += np.sum(p == t[i:i+batch_size])
    
print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

(10000, 784)
(784, 50)
(50, 100)
(100, 10)
Accuracy:0.9352
